In [1]:
# Import libraries
from functions import *

import json
import os
import subprocess
import time

# Creating a P2WPKH transaction

In this section we'll create a P2WPKH transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using bitcoin core in regtest mode.

## Reading
- Andreas Antonopoulos - Mastering Bitcoin Chapter 7
- Jimmy Song - Programming Bitcoin Chapters 13

### Create a P2WPKH UTXO

In order to create a transaction spending from a P2WPKH UTXO, we'll first need to create the UTXO that is locked with a p2wpkh script. To do that, we'll define a private key, use it to generate a pubkey, then convert the pubkey to a p2wpkh address.

#### Create a p2wpkh address 
For more on this step, review the 'Addresses' notebook.

In [2]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
sender_pubkey = privkey_to_pubkey(sender_privkey)
sender_p2wpkh_addr = pk_to_p2wpkh(sender_pubkey, network = "regtest")
print("sender's p2wpkh address: " + sender_p2wpkh_addr)

sender's p2wpkh address: bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw


#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

Now that we have the address, we'll fund it using the python code we used in the previous sections, `create_regtest_utxo.py`.

In [3]:
setup_regtest_bitcoind()
txid_to_spend, index_to_spend = fund_address(sender_p2wpkh_addr, 2.001)
print(f"txid: {txid_to_spend}, {index_to_spend}")

txid: 749c8f8b078898e66f3df5c08403d2fbf9094ac930349821830812246759e662, 0


## Spending a p2wpkh UTXO

Now that we have some funds locked up in a p2wpkh utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw`.

### Decoding a bech32 address

The first thing we need to do is decode the address by doing this we can achieve the following:
1 - validate the checksum to know the address was transmitted without error
2 - make sure we are sending btc on the correct network (testnet/mainnet)
3 - know what to put in the scriptPubkey

For more on addresses, refer back to the 'Addresses' chapter.

In [4]:
receiver_address = 'bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw'

# human readable part
hrp = 'bcrt'
witver, witprog = bech32.decode(hrp, receiver_address)

pubkey_hash = bytearray(witprog)
print("witness version: ", witver)
print("pubkey/script hash: ", pubkey_hash.hex())

witness version:  0
pubkey/script hash:  fc7250a211deddc70ee5a2738de5f07817351cef


The witness version `0`, tells us that this address corresponds to either a p2wpkh or p2wsh output. Then, we can tell from the length of the hash whether it is a pubkey hash (20 bytes long), or a script hash (32 bytes long). 

For more on decoding addresses, refer back to the 'Addresses' chapter.

Now we can create the receiver's output scriptPubkey:

In [5]:
receiver_spk = bytes.fromhex("0014") + pubkey_hash

### Create an unsigned p2wpkh transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [6]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `sender_privkey` and `sender_pubkey`

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("0014") + hash160(change_pubkey)

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the witness field is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [7]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  020000000162e659672412088321983430c94a09f9fbd20384c0f53d6fe69888078b8f9c740000000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d00000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [8]:
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + unsigned_tx.hex())
# print(decoded)

Segwit transactions have a new signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)


In [9]:
pk_hash = hash160(sender_pubkey)
scriptcode = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

input_amount_sat = int(2.001 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print(tx_digest_preimage.hex())

02000000e455836a9ed4458f78270b5f8b12c12a6a5a2d82b4f9c91ca356667070c7c95e3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e7066504462e659672412088321983430c94a09f9fbd20384c0f53d6fe69888078b8f9c74000000001976a914fc7250a211deddc70ee5a2738de5f07817351cef88aca048ed0b00000000fffffffff91a6606c3037951dc241bb6edf31f9a61112940431aee9593c16c3bb3e1d2600000000001000000


Now we are ready to hash this transaction and produce an ecdsa signature on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [10]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Witness field
witness = (
    # indicate the number of stack items for the txin
    # 2 items for signature and pubkey
    bytes.fromhex("02")
    + pushbytes(signature)
    + pushbytes(sender_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  0200000000010162e659672412088321983430c94a09f9fbd20384c0f53d6fe69888078b8f9c740000000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d02473044022100f07c811bf3fa271e81376b228261faed8a4fc334039f220648da829b0758fe6a021f2a5c9ce0cef7fc81b0307637c8f76ad1e0f90f72977e3577ca98a359d1419c0121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa00000000


### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [11]:
new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + signed_tx.hex())
# new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest testmempoolaccept " + "'[\"" +  signed_tx.hex()+ "\"]'")

print(new_tx_txid)

7fb0c54f3c2fca06e2430000f1a143703e4a7a4967b92b6d99d95b5094108252


We can decode the serialized transaction using ```decoderawtransaction```. Notice that our output addresses match the change and receiver addresses from earlier.

In [12]:
print("receiver's p2pkh address: " + receiver_address)
change_p2pkh_addr = pk_to_p2pkh(change_pubkey, network = "regtest")
print("sender's change p2pkh address: " + change_p2pkh_addr)

receiver's p2pkh address: bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw
sender's change p2pkh address: mo6CPsdW8EsnWdmSSCrQ6225VVDtpMBTug


In [13]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + signed_tx.hex())
print(decoded)

{
  "txid": "7fb0c54f3c2fca06e2430000f1a143703e4a7a4967b92b6d99d95b5094108252",
  "hash": "2e0bb03540f812491e7d39a2932faedb6bac672da45f280bfac382ae5aed4b76",
  "version": 2,
  "size": 222,
  "vsize": 141,
  "weight": 561,
  "locktime": 0,
  "vin": [
    {
      "txid": "749c8f8b078898e66f3df5c08403d2fbf9094ac930349821830812246759e662",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3044022100f07c811bf3fa271e81376b228261faed8a4fc334039f220648da829b0758fe6a021f2a5c9ce0cef7fc81b0307637c8f76ad1e0f90f72977e3577ca98a359d1419c01",
        "034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5f078

## Quiz


 ## Answers
    

## Exercise
